# Script to describe the data

In [3]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from utils import misc
import sys
from scipy.stats import skew, kurtosis



root_path = ''
data_path = f'{root_path}data/final_dataset_test.csv'
model_path = f'{root_path}models/'
path_to_images = f'{root_path}images/'
base_log_dir = f'{model_path}logs/'
base_tuning_dir = f'{model_path}tuning/'
results_dir = f'{root_path}results/'
tables_dir = f'{root_path}tables/'

# Specify the desired start and end time
start_time = pd.Timestamp(2019, 10, 31)
end_time = pd.Timestamp(2024, 7, 2)

### Load preprocessed data

In [ ]:
from utils.misc import LoadData

load_data = LoadData()
df, TIME_PERIOD = load_data.load_and_preprocess_data(data_path, start_time, end_time)
#df, time_period = misc.LoadData.load_and_preprocess_data(data_path, start_time, end_time) # Load and preprocess data
print(df.head())

In [ ]:

TIME_START = '2019-10-31'
TIME_END_PERIODS = ['2021-09-30', '2023-01-01', '2024-07-01']
load_data = LoadData()

def calculate_metrics(column):
    metrics = {
        '# of Observations': len(column),
        'Min': column.min(),
        'Max': column.max(),
        'Mean': column.mean(),
        'Standard deviation': column.std(),
        'Skewness': skew(column),
        'Kurtosis': kurtosis(column)
    }
    return metrics

for i in range(len(TIME_END_PERIODS)):
    TIME_END = TIME_END_PERIODS[i]
    df, TIME_PERIOD = load_data.load_and_preprocess_data(data_path, TIME_START, TIME_END)
    # Function to calculate descriptive statistics for a given column


    # Apply the function to each column in the DataFrame
    # Apply the function to the 'Day Ahead Spot Price' column
    results = calculate_metrics(df['Day Ahead Spot Price'])

    # Convert results to a DataFrame by wrapping in a list
    results_df = pd.DataFrame([results])  # Wrapping in a list converts dictionary to one-row DataFrame
    #results_df = results_df.T  # Transpose for readability
    #results_df.columns = ['Day Ahead Spot Price']  # Add column name for clarity

    print(results_df)
    latex_table = results_df.to_latex(buf=f'{tables_dir}day-ahead_prices_from_{TIME_START}_to_{TIME_END}.tex',
                                      index=True, 
                                  caption=f'Descriptive statistics of day-ahead prices from {TIME_START} to {TIME_END}.',
                                  label="table:descriptive_stats",
                                  column_format="lccccccc",
                                  header=True, 
                                  bold_rows=True)
    
    # Generate and save a histogram for the 'Day Ahead Spot Price' column
    plt.figure(figsize=(12, 6))
    #plt.hist(df['Day Ahead Spot Price'], bins=50, color='skyblue', edgecolor='black')
    sns.histplot(df['Day Ahead Spot Price'], bins=100, kde=True, color='skyblue', edgecolor='black')
    plt.title(f"Histogram of Day-Ahead Prices from {TIME_START} to {TIME_END}")
    plt.xlabel("Day-Ahead Price")
    plt.ylabel("Frequency")
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f'{path_to_images}day_ahead_price_histogram_{TIME_START}_to_{TIME_END}.png')
    plt.close()                 

In [ ]:
import pandas as pd
from scipy.stats import skew, kurtosis

# Initialize variables and load_data object
TIME_START = '2019-10-31'
TIME_END_PERIODS = ['2021-09-30', '2023-01-01', '2024-07-01']
load_data = LoadData()

# Function to calculate descriptive statistics for a given column
def calculate_metrics(column):
    metrics = {
        '# of Observations': len(column),
        'Min': column.min(),
        'Max': column.max(),
        'Mean': column.mean(),
        'Standard deviation': column.std(),
        'Skewness': skew(column),
        'Kurtosis': kurtosis(column)
    }
    return metrics

# List to store results for each time period
all_results = []

for i, TIME_END in enumerate(TIME_END_PERIODS):
    df, TIME_PERIOD = load_data.load_and_preprocess_data(data_path, TIME_START, TIME_END)
    
    # Calculate metrics for 'Day Ahead Spot Price' column
    results = calculate_metrics(df['Day Ahead Spot Price'])
    
    # Convert results to a DataFrame and add a column for the time period
    results_df = pd.DataFrame([results])
    results_df.insert(0, 'Time Period', TIME_PERIOD)  # Insert time period as the first column
    print(results_df)
    # Append results to the list
    all_results.append(results_df)

# Concatenate all results into a single DataFrame
combined_results_df = pd.concat(all_results, ignore_index=True)

# Round all numerical columns to two decimal places
combined_results_df = combined_results_df.round(2)

# Round each column to two decimal places and convert to string format
for col in combined_results_df.columns[1:]:  # Skip 'Time Period' column
    combined_results_df[col] = combined_results_df[col].apply(lambda x: f"{x:.2f}")


# Export combined results to LaTeX
latex_table = combined_results_df.to_latex(
    index=False,
    caption="Descriptive statistics of day-ahead prices across different time periods.",
    label="table:combined_descriptive_stats",
    column_format="lccccccc",
    header=True,
    bold_rows=True
)

# Save LaTeX output
with open(f'{tables_dir}day_ahead_descriptive_across_datasets.tex', 'w') as f:
    f.write(latex_table)

print(latex_table)


# Correlation analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming df is your DataFrame
correlation_matrix = df.corr()  # Calculate correlation matrix

# Set the size of the figure
plt.figure(figsize=(8, 4))

# Draw the heatmap with annotation and the coolwarm colormap
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', vmin=-1, vmax=1)

# Display the plot
plt.savefig(f'{path_to_images}correlation/correlation_matrix_{time_period}.png')
plt.show()

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Sample data creation: replace this with your actual Series
# Assume `prices` is your Series with a datetime index
prices = df['Day Ahead Spot Price']

# Resampling the Series into specific time frames
daily_1 = prices.resample('D').mean()  # 1 day
daily_2 = prices.resample('2D').mean()  # 2 days
weekly_1 = prices.resample('W').mean()  # 1 week
weekly_2 = prices.resample('2W').mean()  # 2 weeks
monthly_1 = prices.resample('ME').mean()  # 1 month
monthly_2 = prices.resample('2ME').mean()  # 2 months
monthly_3 = prices.resample('3ME').mean()  # 3 months
quarterly_1 = prices.resample('QE').mean()  # 1 quarter
quarterly_2 = prices.resample('2QE').mean()  # 2 quarters
quarterly_3 = prices.resample('3QE').mean()  # 3 quarters
quarterly_4 = prices.resample('4QE').mean()  # 4 quarters
yearly_1 = prices.resample('YE').mean()  # 1 year
yearly_2 = prices.resample('2YE').mean()  # 2 years
yearly_3 = prices.resample('3YE').mean()  # 3 years
yearly_4 = prices.resample('4YE').mean()  # 4 years

# Combine all resampled series into a DataFrame
data = pd.DataFrame({
    '1D': daily_1,
    '2D': daily_2,
    '1W': weekly_1,
    '2W': weekly_2,
    '1M': monthly_1,
    '2M': monthly_2,
    '3M': monthly_3,
    '1Q': quarterly_1,
    '2Q': quarterly_2,
    '3Q': quarterly_3,
    '4Q': quarterly_4,
    '1Y': yearly_1,
    '2Y': yearly_2,
    '3Y': yearly_3,
    '4Y': yearly_4
})

In [ ]:
# Calculate the correlation matrix
correlation_matrix = data.corr()

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', vmin=-1, vmax=1)

# Add a title
plt.title('Correlation Matrix of Electricity Prices Across Different Time Periods')

# Display the plot
plt.show()
plt.savefig(f'{path_to_images}correlation/correlation_matrix_across_time_{time_period}.png')

# Descriptive statistics

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

print(df.head())
# Calculate mean and confidence intervals
data = df
mean_price = data['Day Ahead Spot Price'].mean()
confidence_interval = 1.96 * data['Day Ahead Spot Price'].std() / np.sqrt(len(data))

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['Day Ahead Spot Price'], label='Electricity Price', color='blue')
plt.fill_between(data.index, mean_price - confidence_interval, mean_price + confidence_interval, color='orange', alpha=0.3, label="95% Confidence Interval")
plt.xlabel("Time")
plt.ylabel("Electricity Price")
plt.legend()
plt.title("Electricity Prices with Confidence Interval")
plt.show()


In [ ]:
import pandas as pd

# Assuming df is already defined and contains 'Day Ahead Spot Price'

# Get the hourly data and calculate statistics
data = df['Day Ahead Spot Price']
hours = pd.date_range('00:00:00', '23:00:00', freq='1h').hour

mean = data.groupby(data.index.hour).mean()
median = data.groupby(data.index.hour).median()
std = data.groupby(data.index.hour).std()
min = data.groupby(data.index.hour).min()
max = data.groupby(data.index.hour).max()
skew = data.groupby(data.index.hour).skew()
kur = data.groupby(data.index.hour).apply(lambda x: x.kurtosis())

lower = data.quantile(0.05)
upper = data.quantile(0.95)
meanclipped = data.clip(lower=lower, upper=upper).groupby(data.index.hour).mean()
skewclipped = data.clip(lower=lower, upper=upper).groupby(data.index.hour).skew()
kurclipped = data.clip(lower=lower, upper=upper).groupby(data.index.hour).apply(lambda x: x.kurtosis())

# Create DataFrame with hours as integers
mean_df = pd.DataFrame({
    'Hour': hours, 
    'Mean': mean, 
    'Mean w/o outliers': meanclipped, 
    'Median': median, 
    'Std': std,
    'Skew': skew, 
    'Skew w/o outliers': skewclipped,
    'Kurtosis': kur,
    'Kurtosis w/o outliers': kurclipped
}).reset_index(drop=True).round(2)

# Export to LaTeX with columns centered except the first one
mean_df.to_latex(buf=f'{tables_dir}day_ahead_descriptive.tex', index=False, 
                 caption="Descriptive statistics of Day Ahead Spot Prices for every hour of day", 
                 label='DayAheadDescriptive', float_format='%.2f', 
                 column_format='l' + 'c' * (len(mean_df.columns) - 1))

mean_df

In [32]:
hours = {}
for i in range(24):
    hours[i] = data.loc[df.index.hour==i]

mean = []
meanclip = []
skew = []
skewclip = []
minimum = []
outlier_low = []
outlier_high = []
maximum = []
spikes_low = []
spikes_high = []
    
for i in [*hours]:
    iqr = hours[i].quantile(.75)-hours[i].quantile(.25)
    outlier_l = hours[i].quantile(.25)-3*iqr
    outlier_h = hours[i].quantile(.75)+3*iqr
    
    mean.append(hours[i].mean())
    skew.append(hours[i].skew())
    minimum.append(hours[i].min())
    outlier_low.append(outlier_l)
    outlier_high.append(outlier_h)
    maximum.append(hours[i].max())
    
    no_spikes_low = (hours[i]<outlier_l).sum()
    no_spikes_high = (hours[i]>outlier_h).sum()
    sh_spikes_low =  no_spikes_low/len(hours[i])
    sh_spikes_high =  no_spikes_high/len(hours[i])
    spikes_low.append(sh_spikes_low*100)
    spikes_high.append(sh_spikes_high*100)
    
    clip = hours[i].loc[(hours[i]>outlier_l) & (hours[i]<outlier_h)]

    meanclip.append(clip.mean())
    skewclip.append(clip.skew())
    
desc = {'Mean': mean, 'Mean w/o outliers': meanclip, 'Skew': skew, 'Skew w/o outliers': skewclip, 'Min': minimum, 'Max': maximum, 'Spikes (%)': np.array(spikes_low)+np.array(spikes_high)}

In [ ]:
import pandas as pd

# Example DataFrame structure; replace with your actual DataFrame containing hourly prices.
# Let's assume your DataFrame is called 'data' and has a DateTimeIndex and a 'price' column.

# Filter the data to match the provided start and end time
start_time = pd.Timestamp(2019, 10, 31)
end_time = pd.Timestamp(2024, 7, 2)
filtered_data = data[(data.index >= start_time) & (data.index <= end_time)]

# Create a list to hold rows for each year
rows = []

# Resample by monthly and quarterly frequencies and calculate the mean of each period
for year in range(start_time.year, end_time.year + 1):
    # Filter for the current year
    year_data = filtered_data[filtered_data.index.year == year]
    
    # Monthly averages (1M, 2M, 3M)
    monthly_avg = year_data.resample('M').mean()
    
    # Quarterly averages (1Q, 2Q, 3Q, 4Q)
    quarterly_avg = year_data.resample('Q').mean()

    # Combine the monthly and quarterly averages
    row = {
        '1M': monthly_avg.iloc[0] if len(monthly_avg) > 0 else None,
        '2M': monthly_avg.iloc[1] if len(monthly_avg) > 1 else None,
        '3M': monthly_avg.iloc[2] if len(monthly_avg) > 2 else None,
        '1Q': quarterly_avg.iloc[0] if len(quarterly_avg) > 0 else None,
        '2Q': quarterly_avg.iloc[1] if len(quarterly_avg) > 1 else None,
        '3Q': quarterly_avg.iloc[2] if len(quarterly_avg) > 2 else None,
        '4Q': quarterly_avg.iloc[3] if len(quarterly_avg) > 3 else None
    }
    
    # Append the row dictionary to the list
    rows.append(pd.DataFrame(row, index=[year]))

# Concatenate all rows into a final DataFrame
result_table = pd.concat(rows)

# Set the index as 'SYS'
result_table.index.name = 'DK1'

# Display the result
print(result_table)

# You can save it to a file if needed
# result_table.to_csv('electricity_price_summary.csv')
result_table.round(2).to_latex(buf=f'{tables_dir}average_traded_prices.tex', 
                               index=True,
                                 caption="Average traded prices across different time periods", 
                                 label='average_traded_prices',
                                 float_format='%.2f',
                                 column_format='l' + 'c' * (len(result_table.columns) - 1))
                 


# Electricity balance

In [ ]:
import requests
start_date = start_time.strftime('%Y-%m-%d') #2015-01-01'
end_date = end_time.strftime('%Y-%m-%d') #'2022-06-01'

price_area  = '{"PriceArea":["DK1"]}'
url = f'https://api.energidataservice.dk/dataset/ElectricityBalanceNonv?offset=0&start={start_date}&end={end_date}&filter=%7B%22PriceArea%22:[%22DK1%22]%7D&sort=HourUTC%20DESC'
response = requests.get(
    url=url)

if response.ok:  # More idiomatic way to check for a successful request
    records = response.json().get('records', [])
    # Directly filtering necessary columns and renaming them
    el_balance = (pd.json_normalize(records).drop(columns=['HourDK', 'PriceArea'])
                          #.loc[:, ['HourUTC', 'SpotPriceDKK']]
                          #.rename(columns={'HourUTC': 'time', 'SpotPriceDKK': 'SpotPriceDK1'})
                          )

    # Convert 'time' column to datetime without timezone information
    #data_el_spot_DK1['time'] = pd.to_datetime(data_el_spot_DK1['time']).dt.tz_localize(None)

    # Display the first few rows of the processed DataFrame
    print(el_balance.head())
else:
    print(f"Failed to fetch data: {response.status_code}")

In [ ]:
# Create DataFrame
#el_balance = pd.DataFrame(data)

# Calculate shares of each energy source with respect to TotalLoad
sources = [
    'Biomass', 'FossilGas', 'FossilHardCoal', 
    'FossilOil', 'HydroPower', 'OtherRenewable', 
    'SolarPower', 'Waste', 'OnshoreWindPower', 
    'OffshoreWindPower'
]

for source in sources:
    el_balance[f'{source}_share'] = (el_balance[source] / el_balance['TotalLoad']) * 100

# Display the modified DataFrame with shares
print(el_balance.head())
el_balance.to_csv(f'{root_path}data/electricity_balance.csv', index=False)

In [ ]:
# Set the HourUTC column as the index for plotting
el_balance['HourUTC'] = pd.to_datetime(el_balance['HourUTC'])
el_balance.set_index('HourUTC', inplace=True)

# Plotting the shares
plt.figure(figsize=(12, 6))
for source in sources:
    plt.plot(el_balance.index, el_balance[f'{source}_share'], marker='', label=source)

plt.title('Shares of Energy Sources with Respect to Total Load')
plt.xlabel('Hour')
plt.ylabel('Share (%)')
plt.xticks(rotation=45)
plt.legend(title='Energy Sources')
plt.grid()
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Calculate monthly averages
monthly_averages = el_balance.resample('ME').mean()

label_mapping = {
    'Biomass': 'Biomass',
    'FossilGas': 'Fossil Gas',
    'FossilHardCoal': 'Fossil Hard Coal',
    'FossilOil': 'Fossil Oil',
    'HydroPower': 'Hydro Power',
    'OtherRenewable': 'Other Renewable',
    'SolarPower': 'Solar Power',
    'Waste': 'Waste',
    'OnshoreWindPower': 'Onshore Wind Power',
    'OffshoreWindPower': 'Offshore Wind Power'
}

# Optional: You can plot the monthly averages if needed
plt.figure(figsize=(12, 6))
for source in sources:
    plt.plot(monthly_averages.index, monthly_averages[f'{source}_share'], marker='', label=label_mapping[source])

plt.title('')
plt.xlabel('')
plt.ylabel('Average Share (%)')
plt.xticks(rotation=45)
plt.legend(title='Energy Sources')
plt.grid()
plt.tight_layout()

# Show the plot
plt.savefig(f'{path_to_images}energy_sources_share.png')
plt.show()